In [0]:
DEBUG = False
CONFIG_FILE = '/datascope/subaru/data/targeting/dSph/ursaminor/priority/ursaminor_nb/ga-priority_20250303214329.config'
OUTPUT_PATH = '/datascope/subaru/data/targeting/dSph/ursaminor/priority/ursaminor_nb/'

# Plot the selection and priority of targets

In [0]:
import os, sys
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import commentjson as json

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Imports

In [0]:
import pfs.utils

from pfs.ga.targeting.scripts.priority.notebooks.notebooks import *
from pfs.ga.targeting.targets.dsph import GALAXIES as DSPH_FIELDS
from pfs.ga.targeting.targets.m31 import M31_FIELDS
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow
from pfs.ga.targeting.io import DataFrameSerializer, ObservationSerializer

# Load the config and args files

In [0]:
# Load the configuration
config = load_priority_config(CONFIG_FILE)

In [0]:
args_file = os.path.splitext(CONFIG_FILE)[0] + '.args'
with open(args_file) as f:
    args = json.load(f)

# Plot definitions

In [0]:
if 'dsph' in args and args['dsph'] is not None:
    field = DSPH_FIELDS[args['dsph']]

hsc = field.get_photometry()
cmd = field.get_cmd()
ccd = field.get_ccd()

In [0]:
pointings = field.get_pointings(SubaruPFI)
pointing = pointings[0]

for p in pointings:
    print(p.ra, p.dec, p.posang)

In [0]:
pfi = SubaruPFI(instrument_options={ 'layout': 'calibration' })
wcs = WcsProjection(Pointing(field.pos), proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

# Load the original and the pre-processed target lists

In [0]:
obs = load_observations(field, config)

obs.data.columns

In [0]:
fn = os.path.expandvars(os.path.join(OUTPUT_PATH, f'hsc_{field.ID}_priorities.feather'))
sample = ObservationSerializer().read(fn)

sample.data.columns

# Plot

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)

axs = [f.add_subplot(gs[0, 0]), f.add_subplot(gs[0, 1]), f.add_subplot(gs[0, 2], projection=wcs.wcs)]

cmd.plot_observation(axs[0], obs, c='gray', observed=True)
ccd.plot_observation(axs[1], obs, c='gray', observed=True)
fov.plot_observation(axs[2], obs, c='gray', observed=True)
        
pp = field.get_pointings(SubaruPFI)
for p in pp:
    pfi.plot_focal_plane(axs[2], fov, corners=True, projection=SubaruWFC(p))

axs[2].set_xlim(0.75, -2.75)
axs[2].set_ylim(-2.75, 0.75)

f.suptitle('HSC targets')

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)

axs = [f.add_subplot(gs[0, 0]), f.add_subplot(gs[0, 1]), f.add_subplot(gs[0, 2], projection=wcs.wcs)]

cmd.plot_observation(axs[0], obs, c='gray', observed=True)
cmd.plot_observation(axs[0], sample, c='red', observed=True)

ccd.plot_observation(axs[1], obs, c='gray', observed=True)
ccd.plot_observation(axs[1], sample, c='red', observed=True)

fov.plot_observation(axs[2], obs, c='gray', observed=True)
fov.plot_observation(axs[2], sample, c='red', observed=True)
        
pp = field.get_pointings(SubaruPFI)
for p in pp:
    pfi.plot_focal_plane(axs[2], fov, corners=True, projection=SubaruWFC(p))

axs[2].set_xlim(0.75, -2.75)
axs[2].set_ylim(-2.75, 0.75)

f.suptitle('HSC targets')

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)

axs = [f.add_subplot(gs[0, 0]), f.add_subplot(gs[0, 1]), f.add_subplot(gs[0, 2], projection=wcs.wcs)]

cmd.plot_observation(axs[0], sample, c=cmap(sample.data['priority']), observed=True)

ccd.plot_observation(axs[1], sample, c=cmap(sample.data['priority']), observed=True)

l = fov.plot_observation(axs[2], sample, c=sample.data['priority'], observed=True, cmap=cmap)
        
pp = field.get_pointings(SubaruPFI)
for p in pp:
    pfi.plot_focal_plane(axs[2], fov, corners=True, projection=SubaruWFC(p))

axs[2].set_xlim(0.75, -2.75)
axs[2].set_ylim(-2.75, 0.75)

# Put a colorbar under all three subplots
f.colorbar(l, ax=axs[2], label='target priority')

f.suptitle('HSC targets')


In [0]:
cmap = plt.get_cmap('viridis')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)

axs = [f.add_subplot(gs[0, 0]), f.add_subplot(gs[0, 1]), f.add_subplot(gs[0, 2], projection=wcs.wcs)]

cmd.plot_observation(axs[0], sample, c=sample.data['exp_time'], observed=True)

ccd.plot_observation(axs[1], sample, c=sample.data['exp_time'], observed=True)

l = fov.plot_observation(axs[2], sample, c=sample.data['exp_time'], observed=True, cmap=cmap)
        
pp = field.get_pointings(SubaruPFI)
for p in pp:
    pfi.plot_focal_plane(axs[2], fov, corners=True, projection=SubaruWFC(p))

axs[2].set_xlim(0.75, -2.75)
axs[2].set_ylim(-2.75, 0.75)

# Put a colorbar under all three subplots
f.colorbar(l, ax=axs[2], label='exposure time')

f.suptitle('HSC targets')
